In [2]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, Reshape, Conv2DTranspose, BatchNormalization, LeakyReLU

In [4]:
# Load data
(train_x, _), (_, _) = fashion_mnist.load_data()
train_x = (train_x / 255.) * 2 - 1
train_x = train_x.reshape(-1, 28, 28, 1)

In [5]:
# Create generator
generator = Sequential([
 Dense(512, input_shape=[100]),
 LeakyReLU(alpha=0.2),
 BatchNormalization(momentum=0.8),
 Dense(256),
 LeakyReLU(alpha=0.2),
 BatchNormalization(momentum=0.8),
 Dense(128),
 LeakyReLU(alpha=0.2),
 BatchNormalization(momentum=0.8),
 Dense(784),
 Reshape([28, 28, 1])
])

In [6]:
# Create discriminator
discriminator = Sequential([
 Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(28, 28, 1)),
 LeakyReLU(alpha=0.2),
 Dropout(0.3),
 Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
 LeakyReLU(alpha=0.2),
 Dropout(0.3),
 Flatten(),
 Dense(1, activation='sigmoid')
])

In [7]:
# Create GAN
GAN = Sequential([generator, discriminator])
discriminator.compile(optimizer='adam', loss='binary_crossentropy')
discriminator.trainable = False
GAN.compile(optimizer='adam', loss='binary_crossentropy')

In [8]:
# Training parameters
epochs = 5
batch_size = 100
noise_shape = 100

In [10]:
# Training loop
for epoch in range(epochs):
 print(f"Currently on Epoch {epoch + 1}")
 for i in range(train_x.shape[0] // batch_size):
   noise = np.random.normal(size=[batch_size, noise_shape])
 gen_image = generator.predict_on_batch(noise)
 train_dataset = train_x[i * batch_size:(i + 1) * batch_size]
 train_label = np.ones(shape=(batch_size, 1))
 discriminator.trainable = True
 d_loss_real = discriminator.train_on_batch(train_dataset, train_label)
 train_label = np.zeros(shape=(batch_size, 1))
 d_loss_fake = discriminator.train_on_batch(gen_image, train_label)
 noise = np.random.normal(size=[batch_size, noise_shape])
 train_label = np.ones(shape=(batch_size, 1))
 discriminator.trainable = False
 d_g_loss_batch = GAN.train_on_batch(noise, train_label)

Currently on Epoch 1
Currently on Epoch 2
Currently on Epoch 3
Currently on Epoch 4
Currently on Epoch 5


In [29]:
# plotting generated images at the start and then after every 10 epoch
if epoch % 10 == 0:
    samples = 10
    x_fake = generator.predict(np.random.normal(loc=0, scale=1, size=(samples, 100)))
    for k in range(samples):
        plt.subplot(2, 5, k + 1)
        plt.imshow(x_fake[k].reshape(28, 28), cmap='gray')
        plt.xticks([])
        plt.yticks([])
        plt.tight_layout()
        plt.show()
        print('Training is complete')

In [ ]:

#Sure, let's break down the provided code line by line:
#
#```python
#import numpy as np
#import matplotlib.pyplot as plt
#from keras.datasets import fashion_mnist
#from keras.models import Sequential
#from keras.layers import Dense, Conv2D, Dropout, Flatten, Reshape, Conv2DTranspose, BatchNormalization, LeakyReLU
#```
#
#- The code begins with importing necessary libraries such as NumPy for numerical computations, Matplotlib for visualization, and Keras for building the neural network model.
#
#```python
## Load data
#(train_x,), (_, _) = fashion_mnist.load_data()
#```
#
#- This line loads the Fashion MNIST dataset. It contains 60,000 training images and 10,000 testing images of 10 different categories of fashion items.
#
#```python
#train_x = (train_x / 255.) * 2 - 1
#```
#
#- The pixel values of the images are normalized to the range [-1, 1] to facilitate training stability.
#
#```python
#train_x = train_x.reshape(-1, 28, 28, 1)
#```
#
#- The shape of the training data is reshaped to include a single channel for grayscale images. The original shape is (60000, 28, 28), and after reshaping, it becomes (60000, 28, 28, 1).
#
#```python
## Create generator
#generator = Sequential([
#    Dense(512, input_shape=[100]),
#    LeakyReLU(alpha=0.2),
#    BatchNormalization(momentum=0.8),
#    Dense(256),
#    LeakyReLU(alpha=0.2),
#    BatchNormalization(momentum=0.8),
#    Dense(128),
#    LeakyReLU(alpha=0.2),
#    BatchNormalization(momentum=0.8),
#    Dense(784),
#    Reshape([28, 28, 1])
#])
#```
#
#- This block defines the generator model, which is a neural network composed of densely connected layers. It takes a 100-dimensional noise vector as input and outputs an image of size 28x28.
#
#```python
## Create discriminator
#discriminator = Sequential([
#    Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(28, 28, 1)),
#    LeakyReLU(alpha=0.2),
#    Dropout(0.3),
#    Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
#    LeakyReLU(alpha=0.2),
#    Dropout(0.3),
#    Flatten(),
#    Dense(1, activation='sigmoid')
#])
#```
#
#- This block defines the discriminator model, which is a convolutional neural network (CNN). It takes an image of size 28x28x1 as input and outputs a single probability value indicating whether the input image is real or fake.
#
#```python
## Create GAN
#GAN = Sequential([generator, discriminator])
#```
#
#- The GAN (Generative Adversarial Network) model is created by stacking the generator on top of the discriminator. This model will be used to train the generator while keeping the discriminator weights frozen.
#
#```python
#discriminator.compile(optimizer='adam', loss='binary_crossentropy')
#```
#
#- The discriminator is compiled with the Adam optimizer and binary cross-entropy loss function. It aims to distinguish between real and fake images.
#
#```python
#discriminator.trainable = False
#GAN.compile(optimizer='adam', loss='binary_crossentropy')
#```
#
#- The discriminator is set to be non-trainable within the GAN model to ensure that only the generator is updated during training.
#
#```python
#epochs = 5
#batch_size = 100
#noise_shape = 100
#```
#
#- Training parameters are defined, including the number of epochs, batch size, and the dimensionality of the noise vector.
#
#```python
#for epoch in range(epochs):
#    print(f"Currently on Epoch {epoch + 1}")
#```
#
#- This loop iterates over each epoch during training and prints the current epoch number.
#
#The rest of the code implements the training loop for the GAN model, where the generator and discriminator are trained alternatively to minimize their respective loss functions. Additionally, it includes visualization of generated images at specific intervals during training.